In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""
#to save openai api ke, which can be used in different scenerios.

Few of the below sections is usefull to handle the scenerio where we need to pass knowledge to gpt-3 as context.

In [ ]:
#this segment can be used to initial a embedding model. Using the sentence transformation, we can achiee a embedding matrix.
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


text ='Abc'
model.encode(text).tolist() #exmple how to do encoding.

In [ ]:
def split_text_into_chunks(plain_text, max_chars=2000):
    text_chunks = []
    current_chunk = ""
    for line in plain_text.split("\n"):
        if len(current_chunk) + len(line) + 1 <= max_chars:
            current_chunk += line + " "
        else:
            text_chunks.append(current_chunk.strip())
            current_chunk = line + " "
    if current_chunk:
        text_chunks.append(current_chunk.strip())
    return text_chunks

#this function is to split the original text in to smaller chunks. As some of the ai models has limitation on size of the data to be passed on at once.

In [ ]:
import pinecone
pinecone.init(api_key="05825101-2760-452e-b534-3508ed2f0a1b", environment="us-east1-gcp") #initializing pinecone, which is basiclly a ector data base. The api_key nd environment can be found from pinecone dashboard.
index = pinecone.Index("gpt-test-embedding") #Initializing with the Index we are using to store the matrix


#Funtion to add data after running any embedding algorithm on it
def addData(corpusData):
    id  = index.describe_index_stats()['total_vector_count']
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.encode(chunk).tolist(), #We are using the model to encode the original chunk of text.
                {'context': chunk}) #In metadata we are storing the original text here as context. 
        index.upsert(vectors=[chunkInfo])

def find_match(query,k):
    query_em = model.encode(query).tolist() #Encoding the text given on a certain search.
    result = index.query(query_em, top_k=k, includeMetadata=True) #Using the encoded text to find the best k matches on the ector database.
    
    return [result['matches'][i]['metadata']['context'] for i in range(k)] #Returning the best results, passing the context, we can also pass other metadata if we want to, i.e title, section etc.
  



In [ ]:
import openai
openai.api_key=os.environ['OPENAI_API_KEY']

# Using the context achieved from the search based on embedding, we'll pass the result as context. This function will generate a prompt out of that. The prompt can be changed according to the need.
def create_prompt(context,query):
    header = "Answer the question as truthfully as possible using the provided context.\n"
    return header + context + "\n\n" + query + "\n"

# Finally, we are requesting to gpt-3 with the generated prompt using this function.
def generate_answer(prompt):
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response.choices[0].text).strip()
     

Some of the blocks below is to do indexing from a big data.

In [ ]:
from gpt_index import Document
#This section is to preapre an array of docs from the splitted chunks in the format gpt index takes the input.
docs = []
text = 'dsadas'
splitted_text = split_text_into_chunks(text)

for x in splitted_text:
  y= Document(x)
  docs.append(y)
print(docs)

In [ ]:
from gpt_index import GPTListIndex, GPTSimpleVectorIndex

index = GPTListIndex(docs) #Passing the generated array of text.

query= 'List all of the articles from the texts.' #Partial Prompt to generate the index

response_structure = "Give the response as an array of object, like, {'title': 'Article 1', 'content': '' }" #Partial prompt to ensure the formt we expect

final_query = query+ '\n' + response_structure #Putting the prompt all together

result = index.query(final_query) #Getting the result from all the text chunks at once! Mke sure to include the openai api ke in the os environment.